In [1]:
import cv2, time, pandas
from datetime import datetime

first_frame = None
status_list = [None, None]
times = []
video = cv2.VideoCapture(0)
df = pandas.DataFrame(columns = ("Start", "End"))

while True:
    check, frame = video.read()
    status = 0
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame,(21,21),0)
    
    if first_frame is None:
        first_frame = gray_frame
        continue
    
    delta_frame = cv2.absdiff(first_frame, gray_frame)
    threshold_frame = cv2.threshold(delta_frame, 30, 255, cv2.THRESH_BINARY)[1]
    threshold_frame = cv2.dilate(threshold_frame, None, iterations = 2)
    (cnts,_) = cv2.findContours(threshold_frame.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in cnts:
        if cv2.contourArea(contour) < 5000:
            continue
        (x, y, w, h) = cv2.boundingRect(contour)
        status = 1
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 3)
    status_list.append(status)
    status_list = status_list[-2:]

    if status_list[-1] == 1 and status_list[-2] == 0:
        times.append(datetime.now())
    if status_list[-1] == 0 and status_list[-2] == 1:
        times.append(datetime.now())
    
    cv2.imshow("Gray Frame", gray_frame)
    cv2.imshow("Delta Frame", delta_frame)
    cv2.imshow("Threshold Frame", threshold_frame)
    cv2. imshow("Color Frame", frame)
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        if status == 1:
            times.append(datetime.now())
        break
for i in range(0, len(times), 2):
    df = df.append({"Start":times[i], "End":times[i+1]}, ignore_index = True)
df.to_csv("Times.csv")

video.release()
cv2.destroyAllWindows()

In [3]:
from bokeh.plotting import figure, show, output_file
from bokeh.io import show, save
from bokeh.models import HoverTool, ColumnDataSource
import pandas

df["Start_string"] = df["Start"].dt.strftime("%Y-%m-%d %H:%M:%S")
df["End_string"] = df["End"].dt.strftime("%Y-%m-%d %H:%M:%S")

cds = ColumnDataSource(df)

fig = figure(x_axis_type = 'datetime', height = 100, width = 500, sizing_mode = "scale_both", title = "Motion Graph")
fig.yaxis.minor_tick_line_color = None
fig.yaxis[0].ticker.desired_num_ticks = 1

hover = HoverTool(tooltips = [("Start","@Start_string"),("End","@End_string")])
fig.add_tools(hover)

q = fig.quad(left = "Start", right = "End", bottom = 0, top = 1, color = "green", source = cds)

output_file("Graph.html")
show(fig)